In [1]:
# !wget https://huggingface.co/datasets/mesolitica/malaysian-ultrachat/resolve/main/ultrachat-crossref-melayu-malay.jsonl

In [19]:
import json
import openai
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import time
from tqdm import tqdm

openai.__version__

'0.28.0'

In [3]:
openai.api_key = ''

In [4]:
openai.api_type = 'azure'
openai.api_base = 'https://husein-ai2-aiservices.openai.azure.com/'
openai.api_version = '2023-03-15-preview'

AZURE_EMBEDDING_ENGINE = 'text-embedding-ada-002'

In [5]:
contexts, questions = [], []

with open('ultrachat-crossref-melayu-malay.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        if l[1]['content_ms']:
            contexts.append(l[0]['content'].strip())
            questions.append(l[1]['content_ms'].strip())
            
len(contexts), len(questions)

(9959, 9959)

In [8]:
contexts_v = []
for i in tqdm(range(len(contexts))):
    while True:
        try:
            r = openai.Embedding.create(input=contexts[i], engine=AZURE_EMBEDDING_ENGINE)
            contexts_v.append(np.array(r.data[0]['embedding']))
            break
        except:
            time.sleep(1.0)

100%|██████████| 9959/9959 [46:03<00:00,  3.60it/s]   


In [9]:
questions_v = []
for i in tqdm(range(len(questions))):
    while True:
        try:
            r = openai.Embedding.create(input=questions[i], engine=AZURE_EMBEDDING_ENGINE)
            questions_v.append(np.array(r.data[0]['embedding']))
            break
        except:
            time.sleep(1.0)

100%|██████████| 9959/9959 [06:01<00:00, 27.52it/s]


In [16]:
contexts_v_np = np.array(contexts_v)
questions_v_np = np.array(questions_v)
contexts_v_np.shape

(9959, 1536)

In [20]:
tops = {
    1: 0,
    3: 0,
    5: 0,
    10: 0,
}

for i in tqdm(range(len(questions_v_np))):
    argsort = np.argsort(cosine_similarity(questions_v_np[i].reshape(1, -1), contexts_v_np)[0])[::-1]
    for k in tops.keys():
        if i in argsort[:k]:
            tops[k] += 1

100%|██████████| 9959/9959 [09:14<00:00, 17.97it/s]


In [21]:
for k, v in tops.items():
    print(k, v / len(questions_v_np))

1 0.3155939351340496
3 0.5120996083944171
5 0.5878100210864544
10 0.6721558389396526
